This code is copy/pasting following the guide here:

https://towardsdatascience.com/text-summarization-from-scratch-using-encoder-decoder-network-with-attention-in-keras-5fa80d12710e

https://gist.github.com/VarunSaravanan?page=1

In [1]:
import numpy as np  
import pandas as pd 
import tensorflow as tf
import re           
import os
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")
import nltk
import os
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from attention import AttentionLayer
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('news_summary_more.csv', encoding='ISO-8859-1', nrows=20000)
df.headlines = df.headlines.apply(lambda x: "sostok " + x + " eostok")

In [3]:
df

,headlines,text
0,sostok upGrad learner switches to career in ML & Al with 90% salary hike eostok,"Saurav Kant, an alumnus of upGrad and IIIT-B's PG Program in Machine learning and Artificial Intelligence, was a Sr Systems Engineer at Infosys with almost 5 years of work experience. The program ..."
1,sostok Delhi techie wins free food from Swiggy for one year on CRED eostok,"Kunal Shah's credit card bill payment platform, CRED, gave users a chance to win free food from Swiggy for one year. Pranav Kaushik, a Delhi techie, bagged this reward after spending 2000 CRED coi..."
2,sostok New Zealand end Rohit Sharma-led India's 12-match winning streak eostok,New Zealand defeated India by 8 wickets in the fourth ODI at Hamilton on Thursday to win their first match of the five-match ODI series. India lost an international match under Rohit Sharma's capt...
3,sostok Aegon life iTerm insurance plan helps customers save tax eostok,"With Aegon Life iTerm Insurance plan, customers can enjoy tax benefits on your premiums paid and save up to Ã¢ÂÂ¹46,800^ on taxes. The plan provides life cover up to the age of 100 years. Also, c..."
4,"sostok Have known Hirani for yrs, what if MeToo claims are not true: Sonam eostok","Speaking about the sexual harassment allegations against Rajkumar Hirani, Sonam Kapoor said, ""I've known Hirani for many years...What if it's not true, the [#MeToo] movement will get derailed."" ""I..."
...,...,...
19995,"sostok Murders of Gauri Lankesh, Narendra Dabholkar linked: CBI eostok",Central Bureau of Investigation on Sunday told a Pune court that it has established links between the murders of Pune-based rationalist Narendra Dabholkar and Bengaluru-based journalist Gauri Lank...
19996,"sostok Nobody respects me, may be they will post my death: Mulayam eostok","Samajwadi Party patriarch Mulayam Singh Yadav on Saturday said, ""Nobody respects me now. May be they will respect me after my death."" He added that even freedom fighter Ram Manohar Lohia had said ..."
19997,sostok Sanskrit teacher arrested for sexually assaulting student eostok,The Delhi Police on Friday arrested a government school's Sanskrit teacher for allegedly sexually assaulting class 10 student during summer vacations in June this year. The crime took place when t...
19998,sostok Islamic State's Afghanistan chief killed in air strike eostok,"Abu Saad Erhabi, the head of Islamic State in Afghanistan, was killed in an air strike in the Nangarhar province on Saturday, the Afghan government said. The strike was part of an operation carrie..."


In [4]:
def decontracted(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text) 
    return text

In [5]:
# preprocessing.py
# Combining all the above we do preprocessing for article text
import re
from tqdm import tqdm
def pre_process(column):
    preprocessed_text = []
    # tqdm is for printing the status bar
    for sentence in tqdm(column.astype(str)):
        sent = decontracted(sentence)
        sent = sent.replace('\\r', ' ')
        sent = sent.replace('\\"', ' ')
        sent = sent.replace('\\n', ' ')
        sent = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", sent)
        sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
        sent=re.sub('<[^>]*>', '', sent)
        preprocessed_text.append(sent.lower().strip())
    return preprocessed_text

In [6]:
df['text']= pre_process(df.text)
df['headlines'] = pre_process(df.headlines)

100%|██████████| 20000/20000 [00:00<00:00, 46174.47it/s]


In [7]:
max_len_text = 62
max_len_headlines = 15

In [8]:
x_tr, x_val, y_tr, y_val = train_test_split(df['text'], df['headlines'], test_size=0.3, random_state=23, shuffle=True)

In [9]:
#Loading our Glove Model 
embeddings_index = dict()
f = open('glove.6B/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

#Calculating Percentage of words from train text present in Word2vec model
words_source_train = []
for i in x_tr :
  words_source_train.extend(i.split(' '))
## Find the total number of words in the Train data of Essays.
print("all the words in the corpus", len(words_source_train))
## Find the unique words in this set of words
words_source_train = set(words_source_train)
print("the unique words in the corpus", len(words_source_train))
## Find the words present in both Glove Vectors as well as our corpus.
inter_words = set(embeddings_index.keys()).intersection(words_source_train)
print("The number of words that are present in both glove vectors and our corpus are {} which \
is nearly {}% ".format(len(inter_words), np.round((float(len(inter_words))/len(words_source_train))
*100)))
words_corpus_source_train = {}
words_glove = set(embeddings_index.keys())
for i in words_source_train:
  if i in words_glove:
    words_corpus_source_train[i] = embeddings_index[i]
print("word 2 vec length", len(words_corpus_source_train))

Loaded 400000 word vectors.
all the words in the corpus 819874
the unique words in the corpus 40598
The number of words that are present in both glove vectors and our corpus are 29540 which is nearly 73.0% 
word 2 vec length 29540


In [10]:
#prepare a tokenizer for text on training data
import pickle
x_tokenizer = Tokenizer()
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences
x_tr    =   x_tokenizer.texts_to_sequences(x_tr )
x_val   =   x_tokenizer.texts_to_sequences(x_val)
#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr,  maxlen=max_len_text, padding='post') 
x_val   =   pad_sequences(x_val, maxlen=max_len_text, padding='post')
x_voc_size   =  len(x_tokenizer.word_index) +1

#preparing a tokenizer for summary on training data 
y_tokenizer = Tokenizer()
y_tokenizer.fit_on_texts(list(y_tr))
#convert summary sequences into integer sequences
y_tr    =   y_tokenizer.texts_to_sequences(y_tr) 
y_val   =   y_tokenizer.texts_to_sequences(y_val) 
#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr, maxlen=max_len_headlines, padding='post')
y_val   =   pad_sequences(y_val, maxlen=max_len_headlines, padding='post')
y_voc_size  =   len(y_tokenizer.word_index) +1

In [11]:
# Embedding Matrix
word_index = x_tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [12]:
#Model

from keras import backend as K 
K.clear_session() 
latent_dim = 500 

# Encoder 
encoder_inputs = Input(shape=(max_len_text,)) 
enc_emb = Embedding(x_voc_size,100,input_length=max_len_text,weights=[embedding_matrix],trainable=False)(encoder_inputs) 

#LSTM 1 
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4) 
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb) 

#LSTM 2 
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4) 
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1) 

#LSTM 3 
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4) 
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2) 

# Set up the decoder. 
decoder_inputs = Input(shape=(None,)) 
dec_emb_layer = Embedding(x_voc_size,100,input_length=max_len_text,weights=[embedding_matrix],trainable=False,) 

dec_emb = dec_emb_layer(decoder_inputs) 

#LSTM using encoder_states as initial state
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.25) 
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c]) 

#Attention Layer
attn_layer = AttentionLayer(name='attention_layer') 
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs]) 

# Concat attention output and decoder LSTM output 
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#Dense layer
decoder_dense = TimeDistributed(Dense(y_voc_size, activation='softmax')) 
decoder_outputs = decoder_dense(decoder_concat_input) 

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs) 
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 62)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 62, 100)      4059900     input_1[0][0]                    
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 62, 500), (N 1202000     embedding[0][0]                  
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
______________________________________________________________________________________________

In [13]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [14]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)
checkpoint = ModelCheckpoint('model_best_weights_news.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', period=1)


In [15]:
history=model.fit([x_tr,y_tr[:,:-1]], y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1)[:,1:] ,epochs=100,batch_size=128, validation_data=([x_val,y_val[:,:-1]], y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]),callbacks = [es,checkpoint])

Epoch 1/100
110/110 [==============================] - ETA: 0s - loss: 6.1120
Epoch 00001: val_loss improved from inf to 5.32862, saving model to model_best_weights_news.h5
110/110 [==============================] - 57s 520ms/step - loss: 6.1120 - val_loss: 5.3286
Epoch 2/100
110/110 [==============================] - ETA: 0s - loss: 5.6657
Epoch 00002: val_loss improved from 5.32862 to 5.21491, saving model to model_best_weights_news.h5
110/110 [==============================] - 57s 517ms/step - loss: 5.6657 - val_loss: 5.2149
Epoch 3/100
110/110 [==============================] - ETA: 0s - loss: 5.4348
Epoch 00003: val_loss improved from 5.21491 to 4.90453, saving model to model_best_weights_news.h5
110/110 [==============================] - 57s 519ms/step - loss: 5.4348 - val_loss: 4.9045
Epoch 4/100
110/110 [==============================] - ETA: 0s - loss: 5.1955
Epoch 00004: val_loss improved from 4.90453 to 4.82645, saving model to model_best_weights_news.h5
110/110 [===========

In [16]:
reverse_target_word_index=y_tokenizer.index_word 
reverse_source_word_index=x_tokenizer.index_word 
target_word_index=y_tokenizer.word_index

In [17]:
# encoder inference
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# decoder inference
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_len_text,latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs)

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat)

# Final decoder model
decoder_model = Model(
[decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
[decoder_outputs2] + [state_h2, state_c2])

In [18]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_len_headlines-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [19]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
      if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
        newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
      if(i!=0):
        newString=newString+reverse_source_word_index[i]+' '
    return newString

In [20]:
for i in range(100):
  print("Review:",seq2text(x_val[i]))
  print("Original summary:",seq2summary(y_val[i]))
  print("Predicted summary:",decode_sequence(x_val[i].reshape(1,max_len_text)))
  print("\n")

Review: the annual of nit trichy is scheduled to take place from to september with this year s theme being an arabian actress of the ok fame will feature in the further artist karan arjun singh who has given sound effects for films like baahubali and wouldangal will be a guest 
Original summary: nit trichy s fest to be held from sep 
Predicted summary:  what to be more in the world s october


Review: salman khangs bodyguard who was guarding former boxing champion mike tyson on his visit to india said that the american boxer ate and liked chicken biryani and took home a traditional sherwani talking about the time he spent with the american boxer said once you work with salman khan handling any celebrity becomes a cakewalk 
Original summary: tyson ate chicken biryani took home a 
Predicted summary:  i have been with female khan on his 1st s death


Review: at least people were killed in zimbabwe after two buses collided on wednesday police said the two buses collided on a road between t